<a href="https://colab.research.google.com/github/Vidusha620/Humming-Bird/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define variables
dataset_folder = '/content/drive/MyDrive/New Data set/New Data set/Train Data Set'
image_height = 480
image_width  = 640
log_dir = os.path.join('Logs')


In [ ]:
# Load dataset
def load_dataset(dataset_folder):
    X = []
    y = []
    class_names = sorted(os.listdir(dataset_folder))
    for class_name in class_names:
        class_dir = os.path.join(dataset_folder, class_name)
        for filename in os.listdir(class_dir):
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (image_height, image_width))
            X.append(img)
            y.append(class_names.index(class_name))

    return np.array(X), np.array(y)

In [ ]:
# Load images and labels
X,y = load_dataset(dataset_folder)


In [ ]:
# Normalize images
X = X / 255.0


In [ ]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
# Define the number of classes dynamically
num_classes = len(os.listdir(dataset_folder))


In [ ]:

# Build CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width,1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train.reshape(-1, image_height, image_width, 1), y_train, epochs=200, validation_data=(X_test.reshape(-1, image_height, image_width, 1), y_test), callbacks=[TensorBoard(log_dir=log_dir)])


Epoch 1/200
12/12 [==============================] - 148s 12s/step - loss: 0.0142 - accuracy: 0.9974 - val_loss: 0.1718 - val_accuracy: 0.9609
Epoch 2/200
12/12 [==============================] - 147s 12s/step - loss: 0.0216 - accuracy: 0.9921 - val_loss: 0.1912 - val_accuracy: 0.9531
Epoch 3/200
12/12 [==============================] - 148s 12s/step - loss: 0.0206 - accuracy: 0.9948 - val_loss: 0.2458 - val_accuracy: 0.9375
Epoch 4/200
12/12 [==============================] - 148s 12s/step - loss: 0.0572 - accuracy: 0.9791 - val_loss: 0.1923 - val_accuracy: 0.9453
Epoch 5/200
12/12 [==============================] - 148s 12s/step - loss: 0.0108 - accuracy: 1.0000 - val_loss: 0.0816 - val_accuracy: 0.9766
Epoch 6/200
12/12 [==============================] - 148s 12s/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0787 - val_accuracy: 0.9844
Epoch 7/200
12/12 [==============================] - 147s 12s/step - loss: 6.0727e-04 - accuracy: 1.0000 - val_loss: 0.1005 - val_accuracy: 0.

KeyboardInterrupt: 

In [ ]:
# Predict labels for test data
y_pred = model.predict(X_test.reshape(-1, image_height, image_width, 1))
y_pred_classes = np.argmax(y_pred, axis=1)


4/4 [==============================] - 20s 5s/step


In [ ]:
# Calculate evaluation metrics
cm = confusion_matrix(y_test, y_pred_classes)
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='macro')
recall = recall_score(y_test, y_pred_classes, average='macro')
f1 = f1_score(y_test, y_pred_classes, average='macro')

In [ ]:
# Print evaluation metrics
print("Confusion Matrix:")
print(cm)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Confusion Matrix:
[[12  0  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  0  0  0  0]
 [ 0  1  0 10  0  0  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0  0]
 [ 0  0  0  0  2  8  0  0  0  0]
 [ 0  0  0  0  0  0 15  0  0  0]
 [ 0  0  0  0  0  0  0  9  0  0]
 [ 0  0  0  0  0  0  0  0 19  0]
 [ 0  0  0  0  0  0  0  0  0 14]]
Accuracy: 0.9765625
Precision: 0.9807142857142856
Recall: 0.9709090909090909
F1 Score: 0.973870573870574


In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 478, 638, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 239, 319, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 237, 317, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 118, 158, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 116, 156, 64)      36928     
                                                                 
 flatten (Flatten)           (None, 1158144)           0

In [ ]:
# Define the path where you want to save the model on Google Drive
model_dir = '/content/drive/MyDrive/Colab Notebooks/Trained Model.h5'


In [ ]:
# Save the model
model.save(os.path.join(model_dir,'Trained Model.h5'))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
